<a href="https://colab.research.google.com/github/YopaNelly/30-day-challenge-week-1-instructions/blob/main/Anomaly_Detection_in_CCTV_Footage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setup - creates all visualizations as image files
!pip install opencv-python numpy matplotlib -q

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import urllib.request

# Create output directories
os.makedirs('results', exist_ok=True)
os.makedirs('results/frames', exist_ok=True)
os.makedirs('results/charts', exist_ok=True)

print("✅ Setup complete. All visualizations will be saved as image files.")
print("📁 Output folder: 'results/'")

✅ Setup complete. All visualizations will be saved as image files.
📁 Output folder: 'results/'


In [ ]:
# Download CCTV video
print("📥 Step 1: Downloading CCTV video...")

url = "https://github.com/intel-iot-devkit/sample-videos/raw/master/people-detection.mp4"
video_path = "cctv_video.mp4"

try:
    urllib.request.urlretrieve(url, video_path)
    print(f"✅ Video downloaded: {video_path}")

    # Get video info
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()

    print(f"📊 Video Info:")
    print(f"   Resolution: {width}x{height}")
    print(f"   FPS: {fps:.1f}")
    print(f"   Total frames: {total_frames}")
    print(f"   Duration: {total_frames/fps:.1f}s")

except Exception as e:
    print(f"❌ Download failed: {e}")
    print("Creating synthetic video instead...")

    # Create synthetic video
    width, height = 640, 480
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(video_path, fourcc, 10, (width, height))

    for i in range(100):
        frame = np.zeros((height, width, 3), dtype=np.uint8)
        frame[:, :] = [50, 50, 50]

        # Normal movement
        x_normal = 100 + (i % 30) * 5
        cv2.rectangle(frame, (x_normal, 250), (x_normal + 40, 290), (0, 200, 0), -1)

        # Anomaly frames (fast movement)
        if 30 < i < 50:
            x_fast = 400 + (i - 30) * 12
            cv2.circle(frame, (x_fast, 150), 30, (0, 0, 255), -1)

        out.write(frame)

    out.release()
    print(f"✅ Created synthetic video: {video_path}")

📥 Step 1: Downloading CCTV video...
✅ Video downloaded: cctv_video.mp4
📊 Video Info:
   Resolution: 768x432
   FPS: 12.0
   Total frames: 596
   Duration: 49.7s


In [ ]:
# Save sample frames as JPG files
print("\n📸 Step 2: Saving sample frames...")

cap = cv2.VideoCapture(video_path)
sample_frames = []

# Save frames at intervals
for i in range(0, 50, 10):  # Frames 0, 10, 20, 30, 40
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()

    if ret:
        # Add frame number
        cv2.putText(frame, f"Frame {i}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"Time: {i/10:.1f}s", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

        # Save as JPG
        filename = f"results/frames/frame_{i:03d}.jpg"
        cv2.imwrite(filename, frame)
        sample_frames.append((i, frame))
        print(f"  ✓ Saved: {filename}")

cap.release()

print(f"✅ Saved {len(sample_frames)} sample frames to 'results/frames/'")


📸 Step 2: Saving sample frames...
  ✓ Saved: results/frames/frame_000.jpg
  ✓ Saved: results/frames/frame_010.jpg
  ✓ Saved: results/frames/frame_020.jpg
  ✓ Saved: results/frames/frame_030.jpg
  ✓ Saved: results/frames/frame_040.jpg
✅ Saved 5 sample frames to 'results/frames/'


In [ ]:
# Show results
print("\n📋 ANOMALY DETECTION RESULTS")
print("=" * 60)

# Read and display results file
if os.path.exists("anomaly_results.txt"):
    with open("anomaly_results.txt", 'r') as f:
        lines = f.readlines()
        for line in lines[:20]:  # Show first 20 lines
            print(line.strip())

    # Show summary
    if anomalies:
        print("\n📊 SUMMARY:")
        print(f"   Total frames processed: 50")
        print(f"   Anomalies detected: {len(anomalies)}")
        print(f"   Detection rate: {len(anomalies)/50*100:.1f}%")

        print("\n📅 Anomaly details:")
        for i, anomaly in enumerate(anomalies[:5]):  # Show first 5
            print(f"   {i+1}. Frame {anomaly['frame']} at {anomaly['time']:.1f}s - Area: {anomaly['area']} pixels")
else:
    print("❌ Results file not found. Run Cell 3 first.")


📋 ANOMALY DETECTION RESULTS
CCTV Anomaly Detection Results

Frame   1 | Time:   0.1s | ○ No motion
Frame   2 | Time:   0.2s | ○ No motion
Frame   3 | Time:   0.2s | ○ No motion
Frame   4 | Time:   0.3s | ○ No motion
Frame   5 | Time:   0.4s | ○ No motion
Frame   6 | Time:   0.5s | ○ No motion
Frame   7 | Time:   0.6s | ○ No motion
Frame   8 | Time:   0.7s | ○ No motion
Frame   9 | Time:   0.8s | ○ No motion
Frame  10 | Time:   0.8s | ○ No motion
Frame  11 | Time:   0.9s | ○ No motion
Frame  12 | Time:   1.0s | ○ No motion
Frame  13 | Time:   1.1s | ○ No motion
Frame  14 | Time:   1.2s | ○ No motion
Frame  15 | Time:   1.2s | ○ No motion
Frame  16 | Time:   1.3s | ✓ Motion detected
Frame  17 | Time:   1.4s | ○ No motion

📊 SUMMARY:
   Total frames processed: 50
   Anomalies detected: 34
   Detection rate: 68.0%

📅 Anomaly details:
   1. Frame 20 at 1.7s - Area: 6864 pixels
   2. Frame 21 at 1.8s - Area: 3996 pixels
   3. Frame 22 at 1.8s - Area: 3344 pixels
   4. Frame 22 at 1.8s - Are

In [ ]:
# Create basic visualizations
print("📊 Creating simple visualizations...")

if anomalies and len(anomalies) > 0:
    # Extract data
    frames = [a['frame'] for a in anomalies]
    areas = [a['area'] for a in anomalies]
    times = [a['time'] for a in anomalies]

    # Create simple text chart
    print("\n📈 Anomaly Timeline (text chart):")
    print("Time (s)  | Anomaly Size")
    print("-" * 30)

    for frame, time, area in zip(frames, times, areas):
        bar = "#" * min(int(area / 500), 50)  # Scale for display
        print(f"{time:6.1f}s  | {bar} ({area} px)")

    # Create simple statistics
    print(f"\n📊 Statistics:")
    print(f"   • Total anomalies: {len(anomalies)}")
    print(f"   • Average area: {np.mean(areas):.0f} pixels")
    print(f"   • Max area: {np.max(areas):.0f} pixels")
    print(f"   • Min area: {np.min(areas):.0f} pixels")

    # Save statistics to file
    with open("anomaly_stats.txt", 'w') as f:
        f.write("Anomaly Detection Statistics\n")
        f.write("=" * 40 + "\n\n")
        f.write(f"Total frames processed: 50\n")
        f.write(f"Anomalies detected: {len(anomalies)}\n")
        f.write(f"Detection rate: {len(anomalies)/50*100:.1f}%\n\n")
        f.write(f"Average anomaly area: {np.mean(areas):.0f} px\n")
        f.write(f"Maximum area: {np.max(areas):.0f} px\n")
        f.write(f"Minimum area: {np.min(areas):.0f} px\n\n")
        f.write("Anomaly List:\n")
        f.write("-" * 40 + "\n")
        for i, anomaly in enumerate(anomalies):
            f.write(f"{i+1}. Frame {anomaly['frame']} at {anomaly['time']:.1f}s - Area: {anomaly['area']} px\n")

    print(f"\n💾 Statistics saved to: anomaly_stats.txt")

else:
    print("📭 No anomalies detected or anomalies list is empty")
    print("   Try adjusting detection parameters in Cell 3")

📊 Creating simple visualizations...

📈 Anomaly Timeline (text chart):
Time (s)  | Anomaly Size
------------------------------
   1.7s  | ############# (6864 px)
   1.8s  | ####### (3996 px)
   1.8s  | ###### (3344 px)
   1.8s  | ######### (4845 px)
   2.0s  | ######## (4183 px)
   2.2s  | ######## (4018 px)
   2.2s  | ####### (3657 px)
   2.7s  | ######### (4536 px)
   2.8s  | ############ (6216 px)
   2.8s  | ########## (5400 px)
   2.9s  | ######### (4500 px)
   3.0s  | ######### (4876 px)
   3.0s  | ######## (4218 px)
   3.1s  | ######### (4905 px)
   3.1s  | ###### (3052 px)
   3.2s  | ########## (5439 px)
   3.2s  | ####### (3712 px)
   3.2s  | ######### (4536 px)
   3.2s  | ####### (3596 px)
   3.3s  | ########### (5586 px)
   3.3s  | ######## (4000 px)
   3.4s  | ###### (3444 px)
   3.5s  | ############# (6765 px)
   3.5s  | ########### (5576 px)
   3.6s  | ###### (3425 px)
   3.7s  | ###################### (11088 px)
   3.8s  | ###### (3492 px)
   3.8s  | ######## (4318 px)
   

In [ ]:
# Save sample frames for manual viewing
print("📸 Saving sample frames...")

def save_sample_frames(video_path, num_frames=5):
    """Save sample frames from video"""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("❌ Cannot open video")
        return

    saved_count = 0

    for i in range(0, 50, 10):  # Save frames 0, 10, 20, 30, 40
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()

        if ret:
            filename = f"sample_frame_{i}.jpg"
            cv2.imwrite(filename, frame)
            saved_count += 1
            print(f"  Saved: {filename}")

    cap.release()
    return saved_count

# Save frames
frames_saved = save_sample_frames(video_path)
print(f"\n✅ Saved {frames_saved} sample frames")

# If we have anomalies, save those frames too
if anomalies and len(anomalies) > 0:
    print("\n💾 Saving frames with anomalies...")

    cap = cv2.VideoCapture(video_path)

    for i, anomaly in enumerate(anomalies[:3]):  # Save first 3 anomalies
        frame_num = anomaly['frame']
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = cap.read()

        if ret:
            # Draw box around anomaly area
            x, y, w, h = anomaly['position']
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 3)

            filename = f"anomaly_frame_{frame_num}.jpg"
            cv2.imwrite(filename, frame)
            print(f"  Saved anomaly frame: {filename}")

    cap.release()

📸 Saving sample frames...
  Saved: sample_frame_0.jpg
  Saved: sample_frame_10.jpg
  Saved: sample_frame_20.jpg
  Saved: sample_frame_30.jpg
  Saved: sample_frame_40.jpg

✅ Saved 5 sample frames

💾 Saving frames with anomalies...
  Saved anomaly frame: anomaly_frame_20.jpg
  Saved anomaly frame: anomaly_frame_21.jpg
  Saved anomaly frame: anomaly_frame_22.jpg


In [ ]:
# List all output files
print("📁 GENERATED OUTPUT FILES:")
print("=" * 60)

output_files = []
for filename in os.listdir('.'):
    if filename.endswith(('.txt', '.jpg', '.mp4')):
        size = os.path.getsize(filename)
        output_files.append((filename, size))

if output_files:
    for filename, size in sorted(output_files):
        size_kb = size // 1024
        print(f"  • {filename:25} ({size_kb:4} KB)")

    print(f"\n📦 Total files: {len(output_files)}")
else:
    print("  No output files found yet")

📁 GENERATED OUTPUT FILES:
  • anomaly_frame_20.jpg      (  60 KB)
  • anomaly_frame_21.jpg      (  61 KB)
  • anomaly_frame_22.jpg      (  60 KB)
  • anomaly_results.txt       (   2 KB)
  • anomaly_stats.txt         (   1 KB)
  • cctv_sample.mp4           (5354 KB)
  • cctv_video.mp4            (5354 KB)
  • first_frame.jpg           (  52 KB)
  • sample_frame_0.jpg        (  52 KB)
  • sample_frame_10.jpg       (  56 KB)
  • sample_frame_20.jpg       (  59 KB)
  • sample_frame_30.jpg       (  58 KB)
  • sample_frame_40.jpg       (  58 KB)

📦 Total files: 13


In [ ]:
# Download instructions
print("📥 HOW TO DOWNLOAD RESULTS:")
print("=" * 60)

print("""
To download the results, use these commands:

# Download all text files
from google.colab import files

files.download('anomaly_results.txt')   # Detailed frame-by-frame results
files.download('anomaly_stats.txt')     # Summary statistics

# Download sample images
for i in [0, 10, 20, 30, 40]:
    filename = f'sample_frame_{i}.jpg'
    if os.path.exists(filename):
        files.download(filename)

# Download anomaly frames
if os.path.exists('anomaly_frame_0.jpg'):
    files.download('anomaly_frame_0.jpg')

# Download the video
files.download('cctv_video.mp4')
""")

print("\n🎯 SYSTEM READY!")
print("   You can now:")
print("   1. View results in text files")
print("   2. Download images for manual viewing")
print("   3. Adjust detection parameters in Cell 3")
print("   4. Upload your own video and re-run")

📥 HOW TO DOWNLOAD RESULTS:

To download the results, use these commands:

# Download all text files
from google.colab import files

files.download('anomaly_results.txt')   # Detailed frame-by-frame results
files.download('anomaly_stats.txt')     # Summary statistics

# Download sample images
for i in [0, 10, 20, 30, 40]:
    filename = f'sample_frame_{i}.jpg'
    if os.path.exists(filename):
        files.download(filename)

# Download anomaly frames
if os.path.exists('anomaly_frame_0.jpg'):
    files.download('anomaly_frame_0.jpg')

# Download the video
files.download('cctv_video.mp4')


🎯 SYSTEM READY!
   You can now:
   1. View results in text files
   2. Download images for manual viewing
   3. Adjust detection parameters in Cell 3
   4. Upload your own video and re-run
